# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secretoss import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/data/CORE/QUOTE/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

In [5]:
data[0].keys()

dict_keys(['avgTotalVolume', 'calculationPrice', 'change', 'changePercent', 'close', 'closeSource', 'closeTime', 'companyName', 'currency', 'delayedPrice', 'delayedPriceTime', 'extendedChange', 'extendedChangePercent', 'extendedPrice', 'extendedPriceTime', 'high', 'highSource', 'highTime', 'iexAskPrice', 'iexAskSize', 'iexBidPrice', 'iexBidSize', 'iexClose', 'iexCloseTime', 'iexLastUpdated', 'iexMarketPercent', 'iexOpen', 'iexOpenTime', 'iexRealtimePrice', 'iexRealtimeSize', 'iexVolume', 'lastTradeTime', 'latestPrice', 'latestSource', 'latestTime', 'latestUpdate', 'latestVolume', 'low', 'lowSource', 'lowTime', 'marketCap', 'oddLotDelayedPrice', 'oddLotDelayedPriceTime', 'open', 'openTime', 'openSource', 'peRatio', 'previousClose', 'previousVolume', 'primaryExchange', 'symbol', 'volume', 'week52High', 'week52Low', 'ytdChange', 'isUSMarketOpen'])

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
price = data[0]['latestPrice']
market_cap = data[0]['marketCap']

In [7]:
price

164.66

In [8]:
market_cap

2605241628360

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [9]:
my_columns = ['ticker', 'price', 'market_cap', 'shares_buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,ticker,price,market_cap,shares_buy


In [10]:
final_dataframe.append(
    pd.Series([symbol,price,market_cap,'N/A'], index=my_columns), ignore_index=True
)

,ticker,price,market_cap,shares_buy
0,AAPL,164.66,2605241628360,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [11]:
my_columns = ['ticker', 'price', 'market_cap', 'shares_buy']
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    try:
        api_url = f'https://api.iex.cloud/v1/data/CORE/QUOTE/{stock}?token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(api_url).json()
        price = data[0]['latestPrice']
        market_cap = data[0]['marketCap']
        final_dataframe = final_dataframe.append(
                pd.Series([stock,price,market_cap,'N/A'], index=my_columns), ignore_index=True)
        # print(stock)
    except:
        print("Error")

In [12]:
final_dataframe

,ticker,price,market_cap,shares_buy
0,A,138.03,40815726770,N/A
1,AAL,14.01,9145959039,N/A
2,AAP,120.69,7153752629,N/A
3,AAPL,164.66,2605241628360,N/A
4,ABBV,161.55,284997105690,N/A
...,...,...,...,...
500,YUM,132.10,37002248702,N/A
501,ZBH,129.17,26993979668,N/A
502,ZBRA,300.26,15433166129,N/A
503,ZION,29.39,4352631315,N/A


In [13]:
final_dataframe.to_csv('final_dataframe.csv')

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [14]:
def chunks(lst,n):
    """Yield succesive n-sized chunks from list"""
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [15]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print('\n', symbol_strings[i])


 A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA

 CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD

 GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,I

In [16]:
for symbol_string in symbol_strings:
    batch_api_call_url = f''

In [17]:
symbol_groups

[0         A
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
       ...  
 95     CINF
 96       CL
 97      CLX
 98      CMA
 99    CMCSA
 Name: Ticker, Length: 100, dtype: object,
 100     CME
 101     CMG
 102     CMI
 103     CMS
 104     CNC
        ... 
 195    FTNT
 196     FTV
 197      GD
 198      GE
 199    GILD
 Name: Ticker, Length: 100, dtype: object,
 200     GIS
 201      GL
 202     GLW
 203      GM
 204    GOOG
        ... 
 295     MAA
 296     MAR
 297     MAS
 298     MCD
 299    MCHP
 Name: Ticker, Length: 100, dtype: object,
 300     MCK
 301     MCO
 302    MDLZ
 303     MDT
 304     MET
        ... 
 395     RHI
 396     RJF
 397      RL
 398     RMD
 399     ROK
 Name: Ticker, Length: 100, dtype: object,
 400     ROL
 401     ROP
 402    ROST
 403     RSG
 404     RTX
        ... 
 495    XLNX
 496     XOM
 497    XRAY
 498     XRX
 499     XYL
 Name: Ticker, Length: 100, dtype: object,
 500     YUM
 501     ZBH
 502    ZBRA
 503    ZION
 504     ZTS
 Name

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [18]:
portfolio_size = input('Enter the value of your portfolio')
try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \nPlease try again:")
    portfolio_size = input('Enter the value of your portfolio')
    val = float(portfolio_size)

In [19]:
position_size = val/len(final_dataframe.index)
print(position_size)

19801.980198019803


In [20]:
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,'shares_buy'] = math.floor(position_size/final_dataframe.loc[i,'price'])

final_dataframe

,ticker,price,market_cap,shares_buy
0,A,138.03,40815726770,143
1,AAL,14.01,9145959039,1413
2,AAP,120.69,7153752629,164
3,AAPL,164.66,2605241628360,120
4,ABBV,161.55,284997105690,122
...,...,...,...,...
500,YUM,132.10,37002248702,149
501,ZBH,129.17,26993979668,153
502,ZBRA,300.26,15433166129,65
503,ZION,29.39,4352631315,673


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [21]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'recommended_trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [22]:
background_color = '#0a0a23'
font_color = '#ffffff'

# creating formats
string_format = writer.book.add_format({'font_color':font_color, 'bg_color':background_color, 'border':1})
dollar_format = writer.book.add_format({'num_format':'$0.00','font_color':font_color, 'bg_color':background_color, 'border':1})
integer_format = writer.book.add_format({'num_format':'0','font_color':font_color, 'bg_color':background_color, 'border':1})


### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [23]:
# writer.sheets['recommended_trades'].set_column('A:A', 18, string_format)
# writer.sheets['recommended_trades'].set_column('B:B', 18, string_format)
# writer.sheets['recommended_trades'].set_column('C:C', 18, string_format)
# writer.sheets['recommended_trades'].set_column('D:D', 18, string_format)
# writer.save()

In [24]:
writer.sheets['recommended_trades'].write('A1','ticker', string_format)
writer.sheets['recommended_trades'].write('B1','price', dollar_format)
writer.sheets['recommended_trades'].write('C1','market_cap', dollar_format)
writer.sheets['recommended_trades'].write('D1','shares_buy', integer_format)

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [25]:
# simplify with loops
column_formats = {
    'A':['ticker', string_format],
    'B':['price', dollar_format],
    'C':['market_cap', dollar_format],
    'D':['shares_buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['recommended_trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['recommended_trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [26]:
writer.save()